In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import utils

In [2]:
sgpc_results = pd.read_csv('results/spgc_coco_results.csv')

In [3]:
# Group the data by book_id and pool_order
grouped = sgpc_results.groupby(['book_id', 'pool_order'])

# Initialize lists to store results
results = []

# Function to calculate WSSR
def calculate_wssr(y_obs, y_pred):
    """
    Weighted Sum of Squared Residuals
    WSSR = Σ (y_observed - y_predicted)^2
    Using equal weights for all points
    """
    return np.sum((y_obs - y_pred)**2 / y_obs**2) # relative error

# Process each group
for (book_id, pool_order), group in grouped:
    lags = group['lag'].values
    abs_coco = np.abs(group['coco_value'].values)
    
    result = {'book_id': book_id, 'pool_order': pool_order}
    
    # Fit power law function
    try:
        popt_pl, _ = curve_fit(
            utils.power_law,
            lags,
            abs_coco,
            bounds=([-np.inf, -np.inf], [np.inf, 0]),
            maxfev=5000
        )
        
        # Calculate predicted values and WSSR
        y_pred_pl = utils.power_law(lags, *popt_pl)
        wssr_pl = calculate_wssr(abs_coco, y_pred_pl)
        
        result.update({
            'pl_a': popt_pl[0],
            'pl_b': popt_pl[1],
            'pl_wssr': wssr_pl
        })
    except RuntimeError:
        pass
    
    # Fit stretched exponential function
    try:
        popt_se, _ = curve_fit(
            utils.stretched_exponential,
            lags,
            abs_coco,
            bounds=([0, 0, -np.inf], [np.inf, 1, np.inf]),
            maxfev=5000
        )
        
        # Calculate predicted values and WSSR
        y_pred_se = utils.stretched_exponential(lags, *popt_se)
        wssr_se = calculate_wssr(abs_coco, y_pred_se)
        
        result.update({
            'se_a': popt_se[0],
            'se_b': popt_se[1],
            'se_c': popt_se[2],
            'se_wssr': wssr_se
        })
    except RuntimeError:
        pass
    
    results.append(result)

# Convert results to DataFrame
fit_results = pd.DataFrame(results)

fit_results.head()

C:\Users\Pawel\AppData\Local\Temp\ipykernel_4008\3902402689.py:25: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_pl, _ = curve_fit(
C:\Users\Pawel\AppData\Local\Temp\ipykernel_4008\3902402689.py:47: OptimizeWarning: Covariance of the parameters could not be estimated
  popt_se, _ = curve_fit(


,book_id,pool_order,pl_a,pl_b,pl_wssr,se_a,se_b,se_c,se_wssr
0,PG10054,0,0.037841,-0.507994,148.839925,3334.410668,0.000152,3331.136132,148.746682
1,PG10054,3,0.351718,-1.082944,32.363354,0.505881,1.000000,-0.562894,47.697232
2,PG10054,9,0.287159,-0.657885,141534.639341,0.159897,1.000000,-1.235222,39.782558
3,PG10054,27,0.184498,-0.449636,1060.756024,0.048947,0.969036,-1.956671,26.882227
4,PG10218,0,0.019864,-0.400116,357.023131,NaN,NaN,NaN,NaN


In [4]:
fit_results.to_csv('results/sgpc_fit_results.csv', index=False)

In [5]:
fit_results = pd.read_csv('results/sgpc_fit_results.csv')

In [6]:
# Create a summary dataframe grouped by pool_order
summary = pd.DataFrame()

# Get total counts per pool_order
total_counts = fit_results.groupby('pool_order').size().reset_index(name='total_count')

# Average SSR values
avg_wssr = fit_results.groupby('pool_order').agg({
    'pl_wssr': 'mean',
    'se_wssr': 'mean'
}).reset_index()

# Count non-null values to determine successful convergence
convergence_counts = fit_results.groupby('pool_order').agg({
    'pl_wssr': 'count',
    'se_wssr': 'count'
}).reset_index()

# Merge all information
summary = total_counts.merge(avg_wssr, on='pool_order')
summary = summary.merge(convergence_counts, on='pool_order', suffixes=('', '_count'))

# Calculate failed counts and percentages
summary['pl_failed_count'] = summary['total_count'] - summary['pl_wssr_count']
summary['se_failed_count'] = summary['total_count'] - summary['se_wssr_count']
summary['pl_failed_percentage'] = (summary['pl_failed_count'] / summary['total_count']) * 100
summary['se_failed_percentage'] = (summary['se_failed_count'] / summary['total_count']) * 100

# Rename columns for clarity
summary.rename(columns={
    'pl_wssr': 'avg_pl_wssr',
    'se_wssr': 'avg_se_wssr',
    'pl_wssr_count': 'pl_success_count',
    'se_wssr_count': 'se_success_count'
}, inplace=True)

# Reorder columns for better readability
summary = summary[['pool_order', 'total_count', 
                  'avg_pl_wssr', 'pl_success_count', 'pl_failed_count', 'pl_failed_percentage',
                  'avg_se_wssr', 'se_success_count', 'se_failed_count', 'se_failed_percentage']]

summary

,pool_order,total_count,avg_pl_wssr,pl_success_count,pl_failed_count,pl_failed_percentage,avg_se_wssr,se_success_count,se_failed_count,se_failed_percentage
0,0,1357,6.799797e+06,1357,0,0.0,3.825908e+06,1203,154,11.348563
1,3,1357,1.398882e+05,1357,0,0.0,1.165149e+05,1273,84,6.190125
2,9,1357,3.546090e+08,1357,0,0.0,5.989441e+03,1356,1,0.073692
3,27,1357,1.074748e+08,1357,0,0.0,3.806977e+05,1357,0,0.000000


In [ ]:
summary_latex = summary[['pool_order', 'pl_failed_percentage', 'avg_pl_wssr', 'se_failed_percentage', 'avg_se_wssr']].to_latex()
print(summary_latex)